In [2]:
!pip install openai azure-cosmos

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 12.5 MB/s eta 0:00:00


In [3]:
import openai
import os
from azure.cosmos import CosmosClient, PartitionKey

In [33]:
from openai import AzureOpenAI
client = AzureOpenAI(
  api_key="L50WlsnegXXqs8HB1vtkbOTCkEXbZJP8aBbJQQJ99AKACYeBjFXJ3w3AAABACOGqLIt",
  api_version = "2024-02-01",
  azure_endpoint = "httpsenai.azure.com/"
)



In [9]:
cosmos_endpoint = 'https://embeddingd.com:443/'
cosmos_key = 'bV0vf8dFeUsWQ2ySrsWkL84dlZRccMjzGF6yWHb5elrz3bGrL73qACDbhZHkqg=='

In [10]:
client = CosmosClient(cosmos_endpoint, cosmos_key)


In [11]:
database_name = "embedcontract"
container_name = "embedcontractcontainer"

In [12]:
database = client.create_database_if_not_exists(id=database_name)
container = database.create_container_if_not_exists(
    id=container_name, partition_key=PartitionKey(path="/label")
)

In [36]:
def generate_embeddings(text):
    response = client.embeddings.create(
        input=text, model="text-embedding-ada-002"
    )
    embeddings = response.data[0].embedding
    return embeddings

In [40]:
labeled_data = [
    {"label": "Termination", "text": "This contract may be terminated by either party."},
    {"label": "Confidentiality", "text": "Confidentiality must be maintained by both parties."},
    {"label": "Indemnification", "text": "The contract shall indemnify both parties from liability."},
    {"label": "Data Sharing", "text": "Data sharing must be limited to both parties only."},
    {"label": "Termination", "text": "Either party can cancel the agreement with notice."},
    # Add more labeled data here

]

Text	Label
"This contract may be terminated by either party."	Termination
"Confidentiality must be maintained by both parties."	Confidentiality
"The contract shall indemnify both parties from liability."	Indemnification
"Data sharing must be limited to both parties only."	Data Sharing
"Either party can cancel the agreement with notice."	Termination
"The confidentiality of shared data is critical."	Confidentiality
"Indemnity provisions are required for both parties."	Indemnification
"Data must not be shared with third parties."	Data Sharing
"Termination is allowed in case of breach of contract."	Termination
"Both parties agree to maintain confidentiality of proprietary information."	Confidentiality
"The contract grants both parties the right to terminate under specific conditions."	Termination
"Any breach in data sharing agreements may lead to legal consequences for both parties."	Data Sharing
"The indemnification clause protects both parties against unforeseen liabilities."	Indemnification
"All data shared between the parties should remain confidential and secure."	Confidentiality
"In case of contract breach, termination may be initiated by the affected party."	Termination
"The parties shall hold each other harmless for damages arising from third-party claims."	Indemnification
"Only relevant data should be shared to avoid unnecessary exposure."	Data Sharing
"This agreement ensures termination rights for both parties under mutual consent."	Termination
"Confidentiality clauses are crucial for protecting sensitive information exchanged."	Confidentiality

In [41]:
# Process and store data
for data_item in labeled_data:
    embeddings = generate_embeddings(data_item["text"])
    document = {
        "id": str(hash(data_item['text'])),
        "label": data_item["label"],
        "text": data_item["text"],
        "embeddings": embeddings,
    }
    container.create_item(body=document)

print("Data stored in Cosmos DB.")

Data stored in Cosmos DB.


In [43]:
from scipy.spatial.distance import cosine
import numpy as np

def find_most_similar(query_text, container):
    # Generate embedding for the query text
    query_embedding = generate_embeddings(query_text)

    # Retrieve all embeddings from Cosmos DB
    items = list(container.read_all_items())

    # Calculate cosine similarity and find the closest match
    similarities = [(item['text'], 1 - cosine(query_embedding, item['embeddings'])) for item in items]
    most_similar = max(similarities, key=lambda x: x[1])

    return most_similar

# Example usage
query_text = "Confidentiality must be preserved."
most_similar_clause = find_most_similar(query_text, container)
print("Most similar clause:", most_similar_clause)

Most similar clause: ('Confidentiality must be maintained by both parties.', 0.9629363670921756)
